# Exploring Hacker News Posts

In this project, we'll work with a data set of submissions to popular technology site [Hacker News](https://news.ycombinator.com/).

Hacker news is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as 'posts') are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

Our [data set](https://www.kaggle.com/hacker-news/hacker-news-posts) has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

We're specifically interested in posts whose titles begin with either 'Ask HN' or 'Show HN'. Users submit 'Ask HN' posts to ask the Hacker News community a specific question. Likewise, users submit 'Show HN' posts to show the Hacker News community a project, product, or just generally something interesting.

We'll compare these two types of posts to determine the following:

- Do Ask HN or Show HN posts receive more comments on average?
- Do posts created at a certain time receive more comments on average?
- Do Ask HN or Show HN posts receive more points on average?
- Do posts created at a certain time receive more points on average?

### Summary of Results

After analyzing the data, we reached the conclusion that Hacker News 'Ask' posts get more comments than 'Show' posts, whereas 'Show' posts get more points than 'Ask' posts. Authors looking to ask a question and get many responses are better off posting between 12p.m. and 4p.m. EST. On the other hand, Hacker News authors that are looking to share one of their creations and get many views are better off posting between 11a.m. and 2p.m. EST.

For more details, please refer to the the full analysis below.

## Reading in the Data

In [1]:
# Read in the data
import csv
opened_file = open('HN_posts_year_to_Sep_26_2016.csv')
read_file = csv.reader(opened_file)
hn = list(read_file)

# Quick exploration of the data
print(hn[:4])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19']]


## Removing Headers

Notice that the first list in the inner lists contains the column headers and the lists after contain the data for one row. In order to analyze our data, we need to first remove the row containing the column headers.

In [2]:
headers = hn[0]

del hn[0]

print(headers, '\n', '\n', hn[:4]) # Exploring the modified dataset

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'] 
 
 [['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19'], ['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16']]


## Filtering our Data

Now that we removed the headers from hn, we're ready to filter our data. Since we're only concerned with posts whose titles begin with 'Ask HN' or 'Show HN', we'll create new lists of lists containing just the data for those titles.

To find the posts that begin with either Ask HN or Show HN, we'll use the string method `startswith`. If we wish to control for case, we can use the `lower` method which returns a lowercase version of the starting string.

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'): # Looking for the titles that belong to the 'Ask HN' category
        ask_posts.append(row)
    elif title.lower().startswith('show hn'): # Looking for the titles that belong to the 'Show HN' category
        show_posts.append(row)
    else:
        other_posts.append(row) # All other posts will be added to a different category
  
# Exploring the filtered dataset 
print(*ask_posts[:3], '\n', 
      *show_posts[:3], '\n', 
      *other_posts[:3], '\n') 

# Checking the number of posts in each category
print('\n Number of ask posts:',len(ask_posts), '\n Number of show posts:', len(show_posts), '\n Number of other posts:', len(other_posts))


['12578908', 'Ask HN: What TLD do you use for local development?', '', '4', '7', 'Sevrene', '9/26/2016 2:53'] ['12578522', 'Ask HN: How do you pass on your work when you die?', '', '6', '3', 'PascLeRasc', '9/26/2016 1:17'] ['12577908', 'Ask HN: How a DNS problem can be limited to a geographic region?', '', '1', '0', 'kuon', '9/25/2016 22:57'] 
 ['12578335', 'Show HN: Finding puns computationally', 'http://puns.samueltaylor.org/', '2', '0', 'saamm', '9/26/2016 0:36'] ['12578182', 'Show HN: A simple library for complicated animations', 'https://christinecha.github.io/choreographer-js/', '1', '0', 'christinecha', '9/26/2016 0:01'] ['12578098', 'Show HN: WebGL visualization of DNA sequences', 'http://grondilu.github.io/dna.html', '1', '0', 'grondilu', '9/25/2016 23:44'] 
 ['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'] ['12579005', 'SQLAR  th

## Analyzing Number of Comments

We will now compute the average number of comments for ask and show posts to determine which category is more popular.

In [4]:
num_comment_ask = 0
num_comment_show = 0
        
for row in ask_posts:
    num_comment_ask += int(row[4])
    
for row in show_posts:
    num_comment_show += int(row[4])

avg_comment_ask = round(num_comment_ask/9138, 2)
avg_comment_show = round(num_comment_show/10158, 2)

print("\n The average number of comments on 'Ask' posts is:", avg_comment_ask, "\n The average number of comments on 'Show' posts is:", avg_comment_show)




 The average number of comments on 'Ask' posts is: 10.39 
 The average number of comments on 'Show' posts is: 4.89


On average, ask posts receive more comments than show posts. Since ask posts are more likely to receive comments, we'll focus the first portion of our analysis just on these posts.

## Analyzing 'Ask' Posts 

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

We will tackle this step by using the `datetime` module to work with the data in the `created_at` column.

In [5]:
import datetime as dt

result_list = []

for row in ask_posts:
    result_list.append([row[6], int(row[4])])
    
counts_by_hour_ask = {}
comments_by_hour = {}

for row in result_list:
    row[0] = dt.datetime.strptime(row[0], '%m/%d/%Y %H:%M')
    hour = dt.datetime.strftime(row[0], '%H')
    if hour not in counts_by_hour_ask:
        counts_by_hour_ask[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour_ask[hour] += 1
        comments_by_hour[hour] += row[1]
        

Here, we have created two dictionaries:

- `counts_by_hour_ask`: contains the number of ask posts created during each hour of the day
- `comments_by_hour`: contains the corresponding number of comments ask posts created at each hour received

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [6]:
avg_by_hour_ask_comm = []

for row in comments_by_hour:
    avg_by_hour_ask_comm.append([row, round(comments_by_hour[row]/counts_by_hour_ask[row], 2)])
    
    

In [7]:
print(*avg_by_hour_ask_comm, sep = '\n')

['02', 11.14]
['01', 7.41]
['22', 8.8]
['21', 8.69]
['19', 7.16]
['17', 9.45]
['15', 28.68]
['14', 9.69]
['13', 16.32]
['11', 8.96]
['10', 10.68]
['09', 6.65]
['07', 7.01]
['03', 7.95]
['23', 6.7]
['20', 8.75]
['16', 7.71]
['08', 9.19]
['00', 7.56]
['18', 7.94]
['12', 12.38]
['04', 9.71]
['06', 6.78]
['05', 8.79]


Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's continue by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [8]:
swap_avg_by_hour = []

for row in avg_by_hour_ask_comm:
    swap_avg_by_hour.append([row[1], row[0]])

print(*swap_avg_by_hour, sep = '\n')

[11.14, '02']
[7.41, '01']
[8.8, '22']
[8.69, '21']
[7.16, '19']
[9.45, '17']
[28.68, '15']
[9.69, '14']
[16.32, '13']
[8.96, '11']
[10.68, '10']
[6.65, '09']
[7.01, '07']
[7.95, '03']
[6.7, '23']
[8.75, '20']
[7.71, '16']
[9.19, '08']
[7.56, '00']
[7.94, '18']
[12.38, '12']
[9.71, '04']
[6.78, '06']
[8.79, '05']


We will now sort this new list in descending order:

In [9]:
sorted_swap_ask = sorted(swap_avg_by_hour, reverse = True)

print(*sorted_swap_ask, sep = '\n')

[28.68, '15']
[16.32, '13']
[12.38, '12']
[11.14, '02']
[10.68, '10']
[9.71, '04']
[9.69, '14']
[9.45, '17']
[9.19, '08']
[8.96, '11']
[8.8, '22']
[8.79, '05']
[8.75, '20']
[8.69, '21']
[7.95, '03']
[7.94, '18']
[7.71, '16']
[7.56, '00']
[7.41, '01']
[7.16, '19']
[7.01, '07']
[6.78, '06']
[6.7, '23']
[6.65, '09']


In [10]:
print('Top 5 Hours for Ask Posts Comments:')

import datetime as dt

for row in sorted_swap_ask[:4]:
    correct_time = dt.datetime.strptime(row[1], '%H')
    correct_time = correct_time.strftime('%H:%M')
    print("{time}: {number} average comments per post".format(time = correct_time, number = row[0]))


Top 5 Hours for Ask Posts Comments:
15:00: 28.68 average comments per post
13:00: 16.32 average comments per post
12:00: 12.38 average comments per post
02:00: 11.14 average comments per post


We notice that the most commented posts are published between 12p.m. and 4p.m. EST (the 3p.m.-4p.m. timeslot being the most popular of all). As an author, if we are looking to publish an ask post on Hacker News and get a higher chance of receiving comments, then this is the timeslot during which we should aim to publish.

## Analyzing Number of Points

We will now compute the average number of points for ask and show posts to determine which category is most upvoted.

In [11]:
num_points_ask = 0
num_points_show = 0

for row in ask_posts:
    num_points_ask += int(row[3])
    
for row in show_posts:
    num_points_show += int(row[3])

avg_points_ask = round(num_points_ask/9138, 2)
avg_points_show = round(num_points_show/10158, 2)

print("\n The average number of points on 'Ask' posts is:", avg_points_ask, "\n The average number of points on 'Show' posts is:", avg_points_show)




 The average number of points on 'Ask' posts is: 11.31 
 The average number of points on 'Show' posts is: 14.84


On average, show posts receive more points than ask posts. Since show posts are more likely to receive points, we'll focus the second portion of our analysis just on these posts.

## Analyzing 'Show' Posts 

Next, we'll determine if show posts created at a certain time are more likely to attract points. We'll use the following steps to perform this analysis:

1. Calculate the amount of show posts created in each hour of the day, along with the number of points received.
2. Calculate the average number of points show posts receive by hour created.

We will tackle this step by using the `datetime` module to work with the data in the `created_at` column.

In [12]:
import datetime as dt

result_list = []

for row in show_posts:
    result_list.append([row[6], int(row[3])])
    
counts_by_hour_show = {}
points_by_hour = {}

for row in result_list:
    row[0] = dt.datetime.strptime(row[0], '%m/%d/%Y %H:%M')
    hour = dt.datetime.strftime(row[0], '%H')
    if hour not in points_by_hour:
        counts_by_hour_show[hour] = 1
        points_by_hour[hour] = row[1]
    else:
        counts_by_hour_show[hour] += 1
        points_by_hour[hour] += row[1]
        

Here, we have created two dictionaries:

- `counts_by_hour_show`: contains the number of show posts created during each hour of the day
- `points_by_hour`: contains the corresponding number of points show posts created at each hour received

Next, we'll use these two dictionaries to calculate the average number of points for posts created during each hour of the day.

In [13]:
avg_by_hour_show_points = []

for row in points_by_hour:
    avg_by_hour_show_points.append([row, round(points_by_hour[row]/counts_by_hour_show[row], 2)])

In [14]:
print(*avg_by_hour_show_points, sep = '\n')

['00', 15.55]
['23', 15.86]
['20', 13.23]
['19', 16.06]
['18', 15.14]
['16', 14.34]
['14', 15.09]
['10', 13.32]
['09', 12.46]
['08', 14.68]
['06', 15.99]
['03', 10.52]
['21', 13.93]
['17', 13.88]
['15', 13.94]
['11', 19.26]
['07', 14.0]
['04', 13.95]
['13', 17.02]
['12', 20.91]
['01', 11.87]
['22', 13.33]
['02', 13.22]
['05', 10.66]


Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's continue by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [15]:
swap_avg_by_hour_show = []

for row in avg_by_hour_show_points:
    swap_avg_by_hour_show.append([row[1], row[0]])

print(*swap_avg_by_hour_show, sep = '\n')

[15.55, '00']
[15.86, '23']
[13.23, '20']
[16.06, '19']
[15.14, '18']
[14.34, '16']
[15.09, '14']
[13.32, '10']
[12.46, '09']
[14.68, '08']
[15.99, '06']
[10.52, '03']
[13.93, '21']
[13.88, '17']
[13.94, '15']
[19.26, '11']
[14.0, '07']
[13.95, '04']
[17.02, '13']
[20.91, '12']
[11.87, '01']
[13.33, '22']
[13.22, '02']
[10.66, '05']


We will now sort this new list in descending order:

In [16]:
sorted_swap_show = sorted(swap_avg_by_hour_show, reverse = True)

print(*sorted_swap_show, sep = '\n')

[20.91, '12']
[19.26, '11']
[17.02, '13']
[16.06, '19']
[15.99, '06']
[15.86, '23']
[15.55, '00']
[15.14, '18']
[15.09, '14']
[14.68, '08']
[14.34, '16']
[14.0, '07']
[13.95, '04']
[13.94, '15']
[13.93, '21']
[13.88, '17']
[13.33, '22']
[13.32, '10']
[13.23, '20']
[13.22, '02']
[12.46, '09']
[11.87, '01']
[10.66, '05']
[10.52, '03']


In [17]:
print('Top 5 Hours for Show Posts Points:')

import datetime as dt

for row in sorted_swap_show[:4]:
    correct_time = dt.datetime.strptime(row[1], '%H')
    correct_time = correct_time.strftime('%H:%M')
    print("{time}: {number} average points per post".format(time = correct_time, number = row[0]))


Top 5 Hours for Show Posts Points:
12:00: 20.91 average points per post
11:00: 19.26 average points per post
13:00: 17.02 average points per post
19:00: 16.06 average points per post


We notice that the most upvoted posts are published between 11a.m. and 2p.m. EST (the 12p.m.-1p.m. timeslot being the most popular of all). As an author, if we are looking to publish a show post on Hacker News and get a higher chance of receiving points, then this is the timeslot during which we should aim to publish.

## Comparing with 'Other' Posts

Given that there are more than 20 times as many 'other' posts as there are 'ask' posts or 'show' posts on Hacker News, we will finish our analysis by comparing the results obtained so far to the average number of comments and points that the 'other' posts receive.

In [18]:
num_comment_other = 0

for row in other_posts:
    num_comment_other += int(row[4])

avg_comment_other = round(num_comment_other/273822, 2)

print("\n The average number of comments on 'other' posts is:", avg_comment_other)

num_points_other = 0

for row in other_posts:
    num_points_other += int(row[3])

avg_points_other = round(num_points_other/273822, 2)

print("\n The average number of points on 'other' posts is:", avg_points_ask)


 The average number of comments on 'other' posts is: 6.46

 The average number of points on 'other' posts is: 11.31


'Other' posts receive a lower amount of comments than 'ask' posts, but receive a higher amount of comments than 'show' posts. On the other hand, they are receiving a higher amount of points than 'ask' posts, but still a lower amount of points than 'show' posts.

For the purpose of this project, we will leave out these results from our conclusion as they do not impact our overall findings.

## Conclusion

The goal of this project was to determine several things:
- Do Ask HN or Show HN posts receive more comments on average?
- Do posts created at a certain time receive more comments on average?
- Do Ask HN or Show HN posts receive more points on average?
- Do posts created at a certain time receive more points on average? 

When comparing the 3 types of posts, we can interpret all the numbers analyzed above as respresentative of the number of unique interactions per post, whether looking at the number of comments or the number of points. It is also important to remind ourselves that 'ask' posts receive the highest amount of comments because of their very nature - they are posts made by users that are asking questions to the Hacker News community and expecting answers (comments) in return.

Therefore, depending on what an author's goal is, there are several options:
- If an author's goal is to ask a question and get many responses, they are better off posting between 12p.m. and 4p.m. EST. This is the publishing timeslot that allows 'ask' posts to get the most comments.
- If an author's goal is to share one of their creations and get many views, they are better off posting between 11a.m. and 2p.m. EST. This is the publishing timeslot that allows 'show' posts to get the most points.